<a href="https://colab.research.google.com/github/GyuhoonK/munhwa_bigdata/blob/master/POS_konlpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Colab을 사용한다면, 이 셀을 실행해주세요
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# 형태소 분석과 텍스트 전처리를 위해 필요한 라이브러리입니다
# 설치되어 있지 않으신 분은 설치해주세요(colab 기본 라이브러리가 아니므로 colab 연결시마다 실행해야합니다)
!pip install -q konlpy 
!pip install -q demoji

In [ ]:
import demoji
demoji.download_codes()

import pickle
import os
import glob

import numpy as np
import pandas as pd
import re
from collections import defaultdict
from tqdm import tqdm_notebook
import datetime

import matplotlib.pyplot as plt

from math import ceil

#DATA_PATH : 기본 경로입니다. ipynb 파일이 저장된 폴더 경로를 지정해주세요
DATA_PATH = '/content/gdrive/My Drive/Colab Notebooks/munhwa_bigdata'
#PKL_PATH : pkl 파일이 저장된 경로를 지정해주세요
PKL_PATH = os.path.join(DATA_PATH, 'pkl')
#POS_PATH : 형태소 분석 결과를 저장할 폴더 경로를 지정해주세요
SAVE_PATH = os.path.join(DATA_PATH, 'pos')

... OK (Got response in 0.12 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [ ]:
def remove_emoji(text):
  return_text = text.lower()
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        "]+", flags = re.UNICODE) 
  return_text = emoji_pattern.sub(r' ', return_text)
  
  return return_text

def remove_date(text):
  date_re = re.compile('\d{4}\.\s\d{1,2}\.\s\d{1,2}\.\s\d{1,2}\:\d{1,2}')
  return_text = date_re.sub(r' ', text)
  return return_text

def remove_char(text):
  char = re.compile('[!@#$%^&*()\-_+=`~[\]/\\,?\\n\\t\;<>|]')
  return_text = char.sub(r' ',text)
  dots = re.compile('\.+')
  return_text = dots.sub(r'.', return_text)
  semidots = re.compile('\,+')
  return_text = semidots.sub(r',', return_text)
  return return_text

def remove_doublespace(text):
  if len(text) == 0 : return ''
  if text[0] == ' ':
    text = text[1:]
  if len(text) == 0 : return ''
  if text[-1] == ' ':
    text = text[:-1]
  return_text = text.replace('  ',' ')
  return return_text
  

In [ ]:
##preprocessing##
def get_removedtext(text):
  removed_text = demoji.replace(text, ' ')
  removed_text = remove_date(removed_text)
  removed_text = remove_char(removed_text)
  removed_text = remove_doublespace(removed_text)
  return removed_text

In [ ]:
def select_pos(tagged_sent, selected_pos):
 return [word for word, pos in tagged_sent if pos in selected_pos]

In [ ]:
def get_rawdocument(document, USE_POS):
  return_docu = ''
  for i in tqdm_notebook(range(len(document))):
    for t in list(map(get_removedtext, document[i])):
      if re.match('\s+', t):
        if re.match('\s+', t).span()  == (0, len(t)) :
          continue
      try:
        result = kkma.pos(t)
        return_docu += ' '.join(select_pos(result, USE_POS)) + ' '
      except :
        pass
  return return_docu

In [ ]:
## 꼬꼬마 형태소 분석기를 사용합니다
from konlpy.tag import Kkma

kkma = Kkma()

In [ ]:
USE_POS = ['NNG', 'NNP', 'VV','VA','XR'] #사용할 품사는 NNG(일반명사), NNP(고유명사), VV(동사), VA(형용사), XR(어근) 입니다

PKL_FILE = glob.glob(os.path.join(PKL_PATH, '*.pkl'))
#아래 세가지 중에 자신이 맡은 부분의 주석을 해제해주세요
###YOUR_PKL = PKL_FILE[:70]
###YOUR_PKL = PKL_FILE[70:140]
###YOUR_PKL = PKL_FILE[140:]

for pkl in tqdm_notebook(YOUR_PKL):
  base = os.path.basename(pkl)
  city = os.path.splitext(base)[0]
  done = list(map(lambda x : os.path.splitext(x)[0], list(map(lambda x : os.path.basename(x) , glob.glob(os.path.join(SAVE_PATH, '*.pkl'))))))
  if city in done :
    continue
  with open(pkl, 'rb') as f:
    bow = pickle.load(f)
  f.close()   

  result = {city:get_rawdocument(bow, USE_POS)}
  with open(os.path.join(SAVE_PATH, '{}.pkl'.format(city)), 'wb') as f:
    pickle.dump(result, f)
  f.close()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until
